In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
from keras import regularizers
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_contrast(image, lower=2.0, upper=1.0)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_hue(image, max_delta = 2.0)
    image = tf.image.random_saturation(image, lower=4.0, upper=2.0)
    return image, label

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, kernel_regularizer=regularizers.l2(0.001),activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, 3,kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(256, 3,kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(256, 3, kernel_regularizer=regularizers.l2(0.001),activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3,kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512, 3, kernel_regularizer=regularizers.l2(0.001),activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 256)       1

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(4).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [9]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [10]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [11]:
history = model.fit(train_data,
                    epochs=20,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 14:35:02.716730: {'loss': 2.9953521738529205, 'accuracy': 0.5368438, 'val_loss': 2.882177706736668, 'val_accuracy': 0.4735}
Epoch 1 ends at 14:39:31.542627: {'loss': 2.44198793053627, 'accuracy': 0.6484875, 'val_loss': 2.32695483857659, 'val_accuracy': 0.5948}
Epoch 2 ends at 14:44:13.825130: {'loss': 2.062612639570236, 'accuracy': 0.69944376, 'val_loss': 2.23206564119667, 'val_accuracy': 0.6359}
Epoch 3 ends at 14:48:59.281428: {'loss': 1.770569336605072, 'accuracy': 0.7385125, 'val_loss': 2.769532773145445, 'val_accuracy': 0.4259}
Epoch 4 ends at 14:53:44.249501: {'loss': 1.4614338208198547, 'accuracy': 0.7731562, 'val_loss': 1.6303976599577885, 'val_accuracy': 0.6882}
Epoch 5 ends at 14:58:31.429775: {'loss': 1.2774457633256913, 'accuracy': 0.798225, 'val_loss': 1.8461781428877715, 'val_accuracy': 0.6299}
Epoch 6 ends at 15:03:16.142416: {'loss': 1.1597538635253906, 'accuracy': 0.8136687, 'val_loss': 1.2992298242392812, 'val_accuracy': 0.7548}
Epoch 7 ends at 15:08:0

In [12]:
model.evaluate(test_data)

157/157 [==============================].9735 - accuracy: 0.81 - 0s 57ms/step - loss: 1.0618 - accuracy: 0.82 - 0s 52ms/step - loss: 1.0639 - accuracy: 0.81 - 0s 50ms/step - loss: 1.0946 - accuracy: 0.79 - 0s 48ms/step - loss: 1.0550 - accuracy: 0.80 - 0s 47ms/step - loss: 1.0192 - accuracy: 0.81 - 0s 46ms/step - loss: 1.0256 - accuracy: 0.80 - 0s 45ms/step - loss: 1.0572 - accuracy: 0.80 - 0s 44ms/step - loss: 1.0468 - accuracy: 0.80 - 0s 44ms/step - loss: 1.0716 - accuracy: 0.80 - 0s 44ms/step - loss: 1.0693 - accuracy: 0.80 - 1s 44ms/step - loss: 1.0782 - accuracy: 0.80 - 1s 44ms/step - loss: 1.0836 - accuracy: 0.80 - 1s 44ms/step - loss: 1.0938 - accuracy: 0.80 - 1s 44ms/step - loss: 1.0932 - accuracy: 0.79 - 1s 44ms/step - loss: 1.0982 - accuracy: 0.79 - 1s 44ms/step - loss: 1.1014 - accuracy: 0.79 - 1s 44ms/step - loss: 1.1158 - accuracy: 0.79 - 1s 44ms/step - loss: 1.1047 - accuracy: 0.79 - 1s 44ms/step - loss: 1.1125 - accuracy: 0.78 - 1s 44ms/step - loss: 1.0992 - accuracy: 0.

[1.14541045191941, 0.7784]

In [13]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))